# Library Test Harness

Quick test of gpxpy and fitparse, using [article](https://towardsdatascience.com/parsing-fitness-tracker-data-with-python-a59e7dc17418) for pointers.

Subsequently added similar tests for pynmeagps and pyubx2.

Created by Michael George (AKA Logiqx)

Website: https://logiqx.github.io/gps-wizard/

In [1]:
import os
import sys
import pprint

import fitdecode
import gpxpy

from pynmeagps import NMEAReader
from pyubx2 import UBXReader

projdir = os.path.realpath(os.path.join(sys.path[0], "..", ".."))

## FIT

Uses [fitdecode](https://pypi.org/project/fitdecode/) as the parser.

In [2]:
def testFit():
    '''FIT test'''
    
    filename = os.path.join(projdir, 'sessions', '20211020', 'Speedsurfing20211020115946.fit')
    
    definitionCounts = 0
    definitionNames = {}
    
    dataCounts = 0
    dataNames = {}
    
    dataRecordCount = 0
    positionalCount = 0

    with fitdecode.FitReader(filename) as fit:
        for frame in fit:
            if frame.frame_type == fitdecode.FIT_FRAME_DEFINITION:
                definitionCounts += 1

                if frame.name in definitionNames:
                    definitionNames[frame.name] += 1
                else:
                    definitionNames[frame.name] = 1

            elif frame.frame_type == fitdecode.FIT_FRAME_DATA:
                dataCounts += 1

                if frame.name in dataNames:
                    dataNames[frame.name] += 1
                else:
                    dataNames[frame.name] = 1

                if frame.name == 'record':
                    dataRecordCount += 1
                        
                    if frame.has_field('position_lat') and frame.get_value('position_lat'):
                        positionalCount += 1
                   
    pp = pprint.PrettyPrinter(indent=4)

    print('FIT FIT_FRAME_DEFINITION\n{}\n'.format(pp.pformat(definitionNames)))

    print('FIT FIT_FRAME_DATA\n{}\n'.format(pp.pformat(dataNames)))
    
    print('FIT data records = {}, positional records = {}'.format(dataRecordCount, positionalCount))

In [3]:
if __name__ == '__main__':
    testFit()
    pass

/opt/conda/lib/python3.9/site-packages/fitdecode/reader.py:909: UserWarning: 'field "native_field_num" (idx #0) not found in message "field_description"' (local_mesg_num: 0; chunk_offset: 198); adding dummy dev data...
  warnings.warn(msg)
/opt/conda/lib/python3.9/site-packages/fitdecode/reader.py:909: UserWarning: 'field "native_field_num" (idx #0) not found in message "field_description"' (local_mesg_num: 0; chunk_offset: 240); adding dummy dev data...
  warnings.warn(msg)
/opt/conda/lib/python3.9/site-packages/fitdecode/reader.py:909: UserWarning: 'field "native_field_num" (idx #0) not found in message "field_description"' (local_mesg_num: 0; chunk_offset: 291); adding dummy dev data...
  warnings.warn(msg)
/opt/conda/lib/python3.9/site-packages/fitdecode/reader.py:909: UserWarning: 'field "native_field_num" (idx #0) not found in message "field_description"' (local_mesg_num: 0; chunk_offset: 311); adding dummy dev data...
  warnings.warn(msg)
/opt/conda/lib/python3.9/site-packages/f

FIT FIT_FRAME_DEFINITION
{   'activity': 1,
    'developer_data_id': 1,
    'device_info': 1,
    'event': 4,
    'field_description': 3,
    'file_id': 1,
    'record': 5,
    'session': 1}

FIT FIT_FRAME_DATA
{   'activity': 1,
    'developer_data_id': 1,
    'device_info': 1,
    'event': 4,
    'field_description': 7,
    'file_id': 1,
    'record': 13107,
    'session': 1}

FIT data records = 13107, positional records = 9520


/opt/conda/lib/python3.9/site-packages/fitdecode/reader.py:584: UserWarning: invalid field size 1 in definition message @ 475889 for type uint32 (expected a multiple of 4)
  warnings.warn(msg)


## GPX

Uses [gpxgy](https://pypi.org/project/gpxpy/) as the parser.

Link to GPX 1.0 [XSD](https://www.topografix.com/gpx/1/0/gpx.xsd)

In [4]:
def testGpx():
    '''GPX test'''
    
    filename = os.path.join(projdir, 'sessions', '20211020', 'Speedsurfing20211020115946.gpx')
    
    with open(filename, 'r') as gpxFile:
        gpx = gpxpy.parse(gpxFile)

    trackPointsCount = 0
    
    counts = \
    {
        'latitude': 0,
        'longitude': 0,
        'time': 0,
        'speed': 0,
        'satellites': 0,
        'horizontal_dilution': 0,
        'course': 0
    }
    
    for track in gpx.tracks:
        for segment in track.segments:
            for point in segment.points:
                trackPointsCount += 1
                for item in counts:
                    if hasattr(point, item):
                        counts[item] += 1
            
    print('GPX points count = {}\n'.format(trackPointsCount))
    
    pp = pprint.PrettyPrinter(indent=4)

    print('GPX point attribute counts\n{}'.format(pp.pformat(counts)))

In [5]:
if __name__ == '__main__':
    testGpx()
    pass

GPX points count = 9520

GPX point attribute counts
{   'course': 9520,
    'horizontal_dilution': 9520,
    'latitude': 9520,
    'longitude': 9520,
    'satellites': 9520,
    'speed': 9520,
    'time': 9520}


## NMEA

Uses [pynmeagps](https://pypi.org/project/pynmeagps/) as the parser.

Link to my [notes](https://logiqx.github.io/gps-guides/nmea.html) about NMEA.

In [6]:
def testNmea():
    '''NMEA test'''
    
    filename = os.path.join(projdir, 'sessions', '20220411', 'GT31_1Hz_GEORG30MICHA_932000175_20220411_111600.nmea')
    
    time = None

    with open(filename, 'rb') as nmeaFile:
        nmr = NMEAReader(nmeaFile, nmeaonly=True)

        timestampsCount = 0

        for (raw_data, parsed_data) in nmr.iterate():
            if parsed_data.msgID in ('RMC', 'GGA'):
                if parsed_data.time != time:
                    time = parsed_data.time
                    timestampsCount += 1

    print('NMEA timestamps count = {}\n'.format(timestampsCount))

In [7]:
if __name__ == '__main__':
    testNmea()
    pass

NMEA timestamps count = 4161



## UBX

Uses [pyubx2](https://pypi.org/project/pyubx2/) as the parser.

In [8]:
def testUbx():
    '''UBX test'''
    
    filename = os.path.join(projdir, 'sessions', '20211020-esp', 'BoomL_83AF2466E48_004.ubx')

    counts = {}

    time = None

    with open(filename, 'rb') as ubxFile:
        nmr = UBXReader(ubxFile, protfilter=2, validate=0)

        timestampsCount = 0
        
        for (raw_data, parsed_data) in nmr.iterate():
            if parsed_data.identity not in counts:
                counts[parsed_data.identity] = 1
            else:
                counts[parsed_data.identity] += 1
                
            if hasattr(parsed_data, 'iTOW'):
                if parsed_data.iTOW != time:
                    time = parsed_data.iTOW

            if parsed_data.identity in ('NAV-PVT'):
                timestampsCount += 1

    print('UBX timestamps count = {}\n'.format(timestampsCount))
    
    pp = pprint.PrettyPrinter(indent=4)

    print('UBX identity counts\n{}'.format(pp.pformat(counts)))

In [9]:
if __name__ == '__main__':
    testUbx()
    pass

UBX timestamps count = 55779

UBX identity counts
{'NAV-PVT': 55779}
